# Convolutional Networks

In [1]:
# Setup
import tensorflow as tf
import keras

import time as timer
import datetime

import os

### Constants

In [2]:
EPOCH = 20
BATCH = 64

DATADIR = "/Users/mghifary/Work/Code/AI/data"
MODELDIR = "/Users/mghifary/Work/Code/AI/models"

modelpath = os.path.join(MODELDIR, "mnist-lenet-classifier.h5")

In [3]:
# Load dataset
data_path = os.path.join(DATADIR, "mnist.npz")
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data(data_path)
x_train = x_train.astype("float32") / 255.
x_test = x_test.astype("float32") / 255.

(n_train, dx1, dx2) = x_train.shape
n_test = x_test.shape[0]

dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
dataset_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))

# Shuffle and batch
dataset = dataset.shuffle(buffer_size=1024).batch(BATCH)
dataset_test = dataset_test.batch(BATCH)

In [4]:
# Build model
# class ConvNet(keras.Sequential):
#     """Multi-layer Perceptron"""

#     def __init__(self, num_classes=10):
#         super().__init__()
#         self.add(keras.layers.Conv2D(8, (3, 3), input_shape=(dx1, dx2, 1)))
#         self.add(keras.layers.MaxPooling2D((2, 2)))
#         self.add(keras.layers.Flatten())
#         self.add(keras.layers.Dense(num_classes))
    
# model = ConvNet(num_classes=10)

class LeNet5(keras.Sequential):
    """LeNet5 architecture"""

    def __init__(self, num_classes=10):
        super().__init__()
        self.add(keras.layers.Conv2D(32, kernel_size=(5, 5), padding="same", activation="relu", input_shape=(dx1, dx2, 1)))
        self.add(keras.layers.MaxPool2D((2, 2)))
        self.add(keras.layers.Conv2D(48, kernel_size=(5, 5), padding="valid", activation="relu"))
        self.add(keras.layers.MaxPool2D((2, 2)))
        self.add(keras.layers.Flatten())
        self.add(keras.layers.Dense(256, activation="relu"))
        self.add(keras.layers.Dense(84, activation="relu"))
        self.add(keras.layers.Dense(num_classes))

model = LeNet5(num_classes=10)
model.build(input_shape=(None, dx1, dx2))
model.summary()

Model: "le_net5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        832       
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 10, 10, 48)        38448     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 48)          0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 1200)              0         
                                                                 
 dense (Dense)               (None, 256)               3074

Train model

In [5]:
# Loss and optimizer
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = keras.optimizers.legacy.Adam(learning_rate=3e-4)


In [6]:
# Define metrics
train_loss = keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = keras.metrics.SparseCategoricalAccuracy('train_accuracy')

test_loss = keras.metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = keras.metrics.SparseCategoricalAccuracy('test_accuracy')

In [7]:
# Training and test function
@tf.function
def train_on_batch(model, loss_fn, optimizer, x, y):
    with tf.GradientTape() as tape:
        y_pred = model(x, training=True)
        loss_value = loss_fn(y, y_pred)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))

    train_loss(loss_value)
    train_accuracy(y, y_pred)

@tf.function
def test_on_batch(model, loss_fn, x, y):
    y_pred = model(x, training=False)
    loss_value = loss_fn(y, y_pred)
    
    test_loss(loss_value)
    test_accuracy(y, y_pred)

In [8]:
# Setup summary writers
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/convnet-mnist/' + current_time + '/train'
test_log_dir = 'logs/convnet-mnist/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [9]:
# Do training
for epoch in range(EPOCH):
    # Mini-batch training
    train_duration = 0.0
    for step, (x, y) in enumerate(dataset):
        start_t = timer.time()
        train_on_batch(model, loss_fn, optimizer, x, y)
        elapsed_t = timer.time() - start_t

        train_duration += elapsed_t
    
    # Store log
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', train_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

    # Test
    test_duration = 0.0
    for step, (xt, yt) in enumerate(dataset_test):
        start_t = timer.time()
        test_on_batch(model, loss_fn, xt, yt)
        elapsed_t = timer.time() - start_t

        test_duration += elapsed_t

    # Store log
    with test_summary_writer.as_default():
        tf.summary.scalar('loss', test_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)

    print(f"Epoch {epoch+1} - Training [loss: {train_loss.result():.5f}, accuracy: {train_accuracy.result():.3f}] ({train_duration:.3f} secs), Test [loss: {test_loss.result():.5f}. accuracy: {test_accuracy.result():.3f}] ({test_duration:.3f} secs)")

    # Save model
    model.save_weights(modelpath, overwrite=True, save_format=None, options=None)

# end of epoch

Epoch 1 - Training [loss: 0.23323, accuracy: 0.931] (12.358 secs), Test [loss: 0.08042. accuracy: 0.975] (0.758 secs)
Epoch 2 - Training [loss: 0.14713, accuracy: 0.956] (12.365 secs), Test [loss: 0.06387. accuracy: 0.980] (0.701 secs)
Epoch 3 - Training [loss: 0.11262, accuracy: 0.966] (12.311 secs), Test [loss: 0.05533. accuracy: 0.982] (0.668 secs)
Epoch 4 - Training [loss: 0.09265, accuracy: 0.972] (12.935 secs), Test [loss: 0.04919. accuracy: 0.984] (0.695 secs)
Epoch 5 - Training [loss: 0.07923, accuracy: 0.976] (13.220 secs), Test [loss: 0.04593. accuracy: 0.985] (0.713 secs)
Epoch 6 - Training [loss: 0.06955, accuracy: 0.979] (12.806 secs), Test [loss: 0.04313. accuracy: 0.986] (0.719 secs)
Epoch 7 - Training [loss: 0.06180, accuracy: 0.981] (13.029 secs), Test [loss: 0.04232. accuracy: 0.986] (0.750 secs)
Epoch 8 - Training [loss: 0.05585, accuracy: 0.983] (13.294 secs), Test [loss: 0.04203. accuracy: 0.987] (0.772 secs)
Epoch 9 - Training [loss: 0.05080, accuracy: 0.985] (13.